In [1]:
import pyomo.environ as pe
import pandas as pd
import numpy as np
import math

## Data Preparation

In [2]:
D = pd.read_csv("Data\d_matrix.csv", index_col = 0)

D

,SF0001,SF0002,SF0003,SF0004,SF0005,SF0006,SF0007,SF0008,SF0009,SF0010,...,SF2014,SF2015,SF2016,SF2017,SF2018,SF2019,SF2020,SF2021,SFA,SFB
node_1,,,,,,,,,,,,,,,,,,,,,
SF0001,0.000,0.432,0.478,0.681,0.576,0.214,0.182,0.106,0.082,0.259,...,44.514,23.531,23.568,23.957,21.169,21.078,28.957,28.895,38.550,13.290
SF0002,0.432,0.000,0.149,0.300,0.145,0.242,0.550,0.369,0.453,0.690,...,44.853,23.734,23.765,24.158,21.600,21.509,29.119,29.068,38.844,13.657
SF0003,0.478,0.149,0.000,0.404,0.174,0.264,0.550,0.389,0.473,0.723,...,44.780,23.615,23.645,24.038,21.614,21.521,28.992,28.943,38.752,13.753
SF0004,0.681,0.300,0.404,0.000,0.241,0.528,0.828,0.644,0.722,0.935,...,45.152,24.019,24.049,24.442,21.829,21.742,29.394,29.346,39.144,13.737
SF0005,0.576,0.145,0.174,0.241,0.000,0.378,0.685,0.507,0.593,0.833,...,44.953,23.783,23.813,24.206,21.742,21.651,29.154,29.107,38.927,13.791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SF2019,21.078,21.509,21.521,21.742,21.651,21.274,20.971,21.144,21.058,20.820,...,29.489,21.719,22.018,22.110,0.434,0.000,27.277,26.675,27.164,13.119
SF2020,28.957,29.119,28.992,29.394,29.154,28.964,28.778,28.910,28.876,28.842,...,24.160,6.045,5.856,5.586,27.708,27.277,0.000,0.803,15.509,34.005
SF2021,28.895,29.068,28.943,29.346,29.107,28.909,28.715,28.851,28.815,28.774,...,23.459,5.704,5.553,5.242,27.107,26.675,0.803,0.000,14.835,33.638


In [3]:
D2 = pd.DataFrame()
for i in D.columns:
    D2[i] = D[i].apply(lambda x: max(x, 5) if x>0 else  (-30/4 + 5))

In [4]:
demand = pd.read_excel("Data\demand.xlsx", index_col = 0)

d = demand.demand
d

customer_code
SF0001      6
SF0002    111
SF0003     37
SF0004     20
SF0005      9
         ... 
SF2017     69
SF2018     63
SF2019     51
SF2020     63
SF2021     82
Name: demand, Length: 2021, dtype: int64

### Model

In [11]:
def gsolver(node, g, g2, d):
    model1 = pe.ConcreteModel()
    model1.x = pe.Var(node, node, domain = pe.Binary)
    model1.y = pe.Var(node, domain = pe.Binary)
    model1.c1 = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.c2 = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.b1 = pe.Var(node, domain=pe.NonNegativeIntegers)
    model1.b2 = pe.Var(node, domain=pe.NonNegativeIntegers)

    model1.costs = pe.Objective(
        expr = 
        sum(model1.x[i,j] * 6 * D.loc[i,j] * model1.c2[i] for i in node for j in node) + 
        sum(model1.x[i,j] * (30+4*(D2.loc[i,j]-5)) * model1.b2[i] for i in node for j in node) + 
        sum(model1.y[j] * 6 * g[j] * model1.c1[j] for j in node) +
        sum(model1.y[j] * (30+4*(g2[j]-5)) * model1.b1[j] for j in node),
        sense = pe.minimize)

    # A node can only be assigned to a hub node
    def rule_1(mod, i, j):
        return mod.x[i,j] <= mod.y[j]
    model1.const1 = pe.Constraint(node, node, rule = rule_1)

    # Each node is assigned to 1 and only 1 hub
    def rule_2(mod, i):
        return sum(mod.x[i, j] for j in node) == 1
    model1.const2 = pe.Constraint(node, rule = rule_2)

    # Each path has at most 2 nodes.
    def rule_5(mod, j):
        return sum(mod.x[i, j] for i in node) <= 2
    model1.const5 = pe.Constraint(node, rule = rule_5)
    
    # Number of Type C2 Vans
    def rule_3(mod, i):
        return 40 * mod.c2[i] + 200 * mod.b2[i] >= d[i]
    model1.const3 = pe.Constraint(node, rule = rule_3)

    # Number of Type C1 Vans
    def rule_4(mod, j):
        return 40 * mod.c1[j] + 200 * mod.b1[j] >= sum(mod.x[i,j] * d[i] for i in node)
    model1.const4 = pe.Constraint(node, rule = rule_4)

    time_limit = 25
    solver = pe.SolverFactory('gurobi')
    solver.options['TimeLimit'] = time_limit
    result = solver.solve(model1, tee = True)
    
    return model1

In [9]:
# original cost
def old_cost(node, i):
    cost = 0
    for a in node:
        original_c = min(D.loc[i][a] * 6 * math.ceil(d[a] / 40), 
                         30 + 4 * (D2.loc[i,a]-5) + D.loc[i][a] * 6 * math.ceil((max(0,d[a]-200)) / 40),
                         2*(30 + 4 * (D2.loc[i,a]-5)) + D.loc[i][a] * 6 * math.ceil((max(0,d[a]-200)) / 40))
        cost += original_c
    return cost

In [12]:
def get_result(cname, result):
    firsts = pd.DataFrame(columns=["local_hub","first_node","C1van"])
    total_c = 0
    total_old_c = 0
    
    nodes = result.sum()
    hub = list(nodes[nodes > 0].index)
    for i in hub:
        node = list(result[result[i]>0][i].index)
        node.remove(i)

        g = D[i]
        g2 = D2[i]
        model1 = gsolver(node, g, g2, d)
        c = model1.costs()
        old_c = old_cost(node, i)
        total_c += c
        total_old_c += old_c

        first = []
        c1van = []
        b1van = []
        for j in node:
            if round(model1.y[j]()) == 1:
                if model1.c1[j]() + model1.b1[j]() > 0:
                    first.append(j)
                    c1van.append(model1.c1[j]())
                    b1van.append(model1.b1[j]())
        first = pd.DataFrame({"local_hub":i, "first_node":first, "C1van":c1van, "B1van":b1van})
        firsts = pd.concat([firsts, first])
        

        for j in node:
            for k in node:
                df1.loc[j][k] = round(model1.x[j,k]())


    total_c = round(total_c,2)
    total_old_c = round(total_old_c, 2)
    print(cname, total_c, total_old_c)
    
    return (total_c, total_old_c, firsts)

In [13]:
%%time

df_cost = pd.DataFrame(columns = ['new_cost', 'old_cost'])
df1 = pd.DataFrame(columns = list(d.index), index = list(d.index))
for n in range(1, 21):
    cname = "cluster" + str(n)
    path = "20abc\\" + cname + "result20bc_node.csv"
    result = pd.read_csv(path, index_col = 0)
    c1, c2, firsts= get_result(cname, result)
    firsts.to_csv(cname + "result_abc_first.csv")
    df_cost.loc[cname] = [c1, c2]
df1.to_csv("result_abc_node.csv")

Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmp4l5232s6.pyomo.lp
Reading time = 0.00 seconds
x67: 61 rows, 67 columns, 205 nonzeros
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 61 rows, 67 columns and 205 nonzeros
Model fingerprint: 0x4c9dac5c
Model has 72 quadratic objective terms
Variable types: 1 continuous, 66 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [3e-01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 7.497360e+11
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 204 rows, 282 c

    containing a solution
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmpnex1q59p.pyomo.lp
Reading time = 0.00 seconds
x205: 193 rows, 205 columns, 769 nonzeros
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 193 rows, 205 columns and 769 nonzeros
Model fingerprint: 0x439e6530
Model has 288 quadratic objective terms
Variable types: 1 continuous, 204 integer (156 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 1.759056e+12
Presolve removed 1 rows and 1 columns
Presolve time

    containing a solution
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmpm29cki__.pyomo.lp
Reading time = 0.01 seconds
x415: 397 rows, 415 columns, 1693 nonzeros
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 397 rows, 415 columns and 1693 nonzeros
Model fingerprint: 0xe2e4ebe6
Model has 648 quadratic objective terms
Variable types: 1 continuous, 414 integer (342 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 2.195388e+12
Presolve removed 1 rows and 1 columns
Presolve ti

    containing a solution
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmp1h6bkbnt.pyomo.lp
Reading time = 0.01 seconds
x1185: 1153 rows, 1185 columns, 5249 nonzeros
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1153 rows, 1185 columns and 5249 nonzeros
Model fingerprint: 0x049dff27
Model has 2044 quadratic objective terms
Variable types: 1 continuous, 1184 integer (1056 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-02, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]
Found heuristic solution: objective 3.988776e+12
Presolve removed 1 rows and 1 columns
Pre

H  895   890                      58.5720000    0.00000   100%   6.2    0s
H 1634  1232                      57.8220000   26.66235  53.9%   7.1    0s
H 1636  1171                      57.8100000   27.17748  53.0%   7.1    0s
H 1636  1112                      56.4120000   27.32337  51.6%   7.1    0s
H 1637  1058                      55.5120000   27.64461  50.2%   7.1    0s
H 1639  1006                      54.5520000   27.72040  49.2%   7.1    0s
H 1639   955                      54.1440000   27.72040  48.8%   7.1    0s
H 1643   910                      53.8980000   31.92556  40.8%   7.1    0s
H 1645   865                      53.1360000   32.91051  38.1%   7.0    0s
H 1645   822                      51.4920000   33.44014  35.1%   7.0    0s
H 1654   786                      50.8080000   39.46467  22.3%   7.0    1s
H 1677   762                      50.7060000   44.86958  11.5%   7.9    1s
H 1682   726                      50.6880000   46.20766  8.84%   7.9    1s
H 1873   729             

...

    containing a solution
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmpozqsqerc.pyomo.lp
Reading time = 0.01 seconds
x457: 438 rows, 457 columns, 1882 nonzeros
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 438 rows, 457 columns and 1882 nonzeros
Model fingerprint: 0xc7494e00
Model has 722 quadratic objective terms
Variable types: 1 continuous, 456 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 2.721360e+12
Presolve removed 1 rows and 1 columns
Presolve ti

    containing a solution
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmp8esdamsj.pyomo.lp
Reading time = 0.01 seconds
x301: 286 rows, 301 columns, 1186 nonzeros
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 286 rows, 301 columns and 1186 nonzeros
Model fingerprint: 0x7ca2ecc4
Model has 450 quadratic objective terms
Variable types: 1 continuous, 300 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 1.903572e+12
Presolve removed 1 rows and 1 columns
Presolve ti

    containing a solution
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmpkmuebpl5.pyomo.lp
Reading time = 0.01 seconds
x205: 193 rows, 205 columns, 769 nonzeros
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 193 rows, 205 columns and 769 nonzeros
Model fingerprint: 0x8bf783f8
Model has 288 quadratic objective terms
Variable types: 1 continuous, 204 integer (156 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 1.691292e+12
Presolve removed 1 rows and 1 columns
Presolve time

    containing a solution
Using license file C:\Users\Yishu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-20
Read LP format model from file C:\Users\Yishu\AppData\Local\Temp\tmp8avn7wp_.pyomo.lp
Reading time = 0.01 seconds
x337: 321 rows, 337 columns, 1345 nonzeros
Changed value of parameter TimeLimit to 25.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 321 rows, 337 columns and 1345 nonzeros
Model fingerprint: 0xe08169c7
Model has 512 quadratic objective terms
Variable types: 1 continuous, 336 integer (272 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 2.286696e+12
Presolve removed 1 rows and 1 columns
Presolve ti

    containing a solution
cluster20 581.74 669.04
Wall time: 51min 16s


In [14]:
df_cost

,new_cost,old_cost
cluster1,774.68,885.14
cluster2,1048.28,1132.48
cluster3,1224.30,1354.10
cluster4,868.40,1013.89
cluster5,842.12,927.02
cluster6,805.61,964.04
cluster7,820.56,888.60
cluster8,864.61,976.77
cluster9,1196.10,1345.06
cluster10,639.37,706.21


In [15]:
df_cost.sum()

new_cost    17830.88
old_cost    19941.76
dtype: float64